In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Считывание данных
df = pd.read_csv('data.csv')

# Вывод информации о данных
print("Информация о данных:")
print(f"Количество строк: {df.shape[0]}")
print(f"Количество столбцов: {df.shape[1]}")
print("\nТипы данных по столбцам:")
print(df.dtypes)

Информация о данных:
Количество строк: 33725
Количество столбцов: 32

Типы данных по столбцам:
hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                            int64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes        

In [10]:
# Выборка признаков и целевой переменной
features = ['hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'adults', 'children', 'country']
target = 'is_canceled'

# Определение целевой переменной и признаков
X = df[features]
y = df[target]

# Преобразование категориальных признаков в числовые значения
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    X.loc[:, column] = label_encoders[column].fit_transform(X.loc[:, column])
        
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Замена отсутсвующих и бесконечных значений на ноль
X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
X_test = X_test.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



# Настройка гиперпараметров для решающего дерева
params_dt = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

decision_tree = DecisionTreeClassifier(random_state=42)
grid_search_dt = GridSearchCV(decision_tree, params_dt, cv=5)
grid_search_dt.fit(X_train_scaled, y_train)

best_decision_tree = grid_search_dt.best_estimator_
predictions_dt_tuned = best_decision_tree.predict(X_test_scaled)
accuracy_dt_tuned = accuracy_score(y_test, predictions_dt_tuned)

print("Лучшие параметры для решающего дерева:", grid_search_dt.best_params_)
print("Точность настроенных параметров решающего дерева:", accuracy_dt_tuned)
print("Classification Report of Tuned Decision Tree:")
print(classification_report(y_test, predictions_dt_tuned))

params_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1]
}

random_forest = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(random_forest, params_rf, cv=5)
grid_search_rf.fit(X_train_scaled, y_train)

best_random_forest = grid_search_rf.best_estimator_
predictions_rf_tuned = best_random_forest.predict(X_test_scaled)
accuracy_rf_tuned = accuracy_score(y_test, predictions_rf_tuned)

print("\nЛушчие параметры для случайного леса:", grid_search_rf.best_params_)
print("Точность настроенных параметров случайного леса:", accuracy_rf_tuned)
print("Classification Report of Tuned Random Forest:")
print(classification_report(y_test, predictions_rf_tuned))

Лучшие параметры для решающего дерева: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
Точность настроенных параметров решающего дерева: 0.8443291326908822
Classification Report of Tuned Decision Tree:
              precision    recall  f1-score   support

           0       0.85      0.93      0.89      4486
           1       0.84      0.66      0.74      2259

    accuracy                           0.84      6745
   macro avg       0.84      0.80      0.81      6745
weighted avg       0.84      0.84      0.84      6745


Лушчие параметры для случайного леса: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Точность настроенных параметров случайного леса: 0.8704225352112676
Classification Report of Tuned Random Forest:
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      4486
           1       0.83      0.77      0.80      2259

    accuracy                           0.87     